In [1]:
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive
from geopy import geocoders  
from geopy.geocoders import Nominatim
from datetime import datetime
from astropy.coordinates import EarthLocation, AltAz, SkyCoord
from astropy.time import Time

In [2]:
def julian_date(date_time):
    jd = date_time.toordinal() + 1721424.5 + (date_time.hour + date_time.minute / 60 + date_time.second / 3600) / 24
    return jd

In [3]:
def gst_from_jd(jd):
    '''
    Calculate Greenwich Sidereal Time (GST) from Julian Date.
    '''
    T = (jd - 2451545.0) / 36525
    GST = 280.46061837 + 360.98564736629 * (jd - 2451545) + T**2 * (0.000387933 - T / 38710000)
    GST = GST % 360 # Normalize to 0 - 360 degrees
    return GST

In [4]:
def lst_from_gst(gst, longitude):
    """
    Calculate Local Sidereal Time (LST) from Greenwich Sidereal Time (GST) and longitude.
    """
    LST = gst + longitude
    LST = LST % 360  # Normalize to 0 - 360 degrees
    return LST

In [5]:
def zenith_ra_dec(latitude, longitude, date_time):
    """
    Compute the Right Ascension (RA) and Declination (Dec) of the zenith for a given location and date.
    """
    jd = julian_date(date_time)
    gst = gst_from_jd(jd)
    lst = lst_from_gst(gst, longitude)
    dec = latitude
    ra = lst
    return ra, dec

In [6]:
def map_eccentricity_to_trait(eccentricity):
    if eccentricity == 0:
        return "are perfectly stable"
    elif 0 < eccentricity < 0.3:
        return "prefer stability"
    elif 0.3 <= eccentricity < 0.6:
        return "are balanced"
    elif 0.6 <= eccentricity < 0.9:
        return "prefer excitement"
    else:
        return "embrace change"

def map_semimajor_axis_to_trait(semimajor_axis):
    if semimajor_axis < 0.1:
        return "are 'close to the action' and constantly influenced by your star's energy, suggesting a very outgoing and active nature"
    elif 0.1 <= semimajor_axis < 1:
        return "are still within a region of significant stellar influence, indicating a generally social and engaging character"
    elif 1 <= semimajor_axis < 5:
        return "strike a balance between the inner and outer regions, reflecting a well-rounded personality that is equally comfortable in social situations and solitude"
    elif 5 <= semimajor_axis < 30:
        return "are farther from the star, implying a more reserved and introspective nature, preferring less direct interaction"
    else:
        return "are on the outskirts, indicating a highly introspective and solitary disposition, thriving in their own space away from the hustle and bustle"

In [7]:
def map_orbital_period_to_trait(orbital_period):
    if orbital_period < 10:
        return "rapid orbits suggest a fast-paced and reactive thinking style"
    elif 10 <= orbital_period < 100:
        return "orbits allow for rapid changes and adaptation, indicating an active and adaptive thinking style"
    elif 100 <= orbital_period < 365:
        return "orbital periods tend to experience balanced conditions, suggesting a balanced and analytical thinking style"
    elif 365 <= orbital_period < 3650:
        return "planets take longer to orbit their stars, implying a more deliberate and thoughtful approach"
    else:
        return "very long orbital periods embody a reflective and contemplative thinking style"

def map_stellar_mass_to_trait(stellar_mass):
    if stellar_mass < 0.5:
        return "stable and enduring"
    elif 0.5 <= stellar_mass < 1.5:
        return "balanced and nurturing"
    elif 1.5 <= stellar_mass < 3:
        return "dynamic and charismatic"
    else:
        return "intense and transformative"

In [8]:
def generate_horoscope_message(user, year, month, day, hour, minute, second, citystate):
    geolocator = Nominatim(user_agent='moeur')
    location = geolocator.geocode(citystate)
    date_time = datetime(year, month, day, hour, minute, second)
    birth_ra, birth_dec = location[1][0], location[1][1]
    zen_ra, zen_dec = zenith_ra_dec(birth_ra, birth_dec, date_time)
    table = NasaExoplanetArchive.query_region(
    table="ps", coordinates=SkyCoord(zen_ra * u.deg, zen_dec * u.deg),
    radius=24 * u.deg) 
    coords = SkyCoord(zen_ra * u.deg, zen_dec * u.deg)
    stars_coords = SkyCoord(table['ra'], table['dec'], unit=(u.deg, u.deg))
    distances = coords.separation(stars_coords)
    closest_index = distances.argmin()
    closest_object = table[closest_index]
    closest_table = NasaExoplanetArchive.query_object(closest_object['pl_name'])
    name = closest_table['pl_name'][0]
    star = closest_table['hostname'][0]
    eccentricity = np.nanmean(closest_table["pl_orbeccen"])
    semi_major_axis = np.nanmean(np.asarray(closest_table["pl_orbsmax"].value))
    period = np.nanmean(np.asarray(closest_table["pl_orbper"].value))
    stellar_mass = np.nanmean(np.asarray(closest_table["st_mass"].value))
    
    eccentricity_trait = map_eccentricity_to_trait(eccentricity)
    axis_trait = map_semimajor_axis_to_trait(semi_major_axis)
    period_trait = map_orbital_period_to_trait(period)
    stellar_mass_trait = map_stellar_mass_to_trait(stellar_mass)
    
    message = (f"{user}, your birth exoplanet is {name} orbiting star {star}. "
               f"Based on an eccentricity of {eccentricity:.2f}, you {eccentricity_trait}. "
               f"With an orbit semi-major axis of {semi_major_axis:.2f} AU, you {axis_trait}. "
               f"With a birth exoplanet period of {period:.2f} days, these {period_trait}, "
               f"and with a stellar mass of {stellar_mass:.2f} solar masses, you are {stellar_mass_trait}.")
    
    return message


In [9]:
generate_horoscope_message('Chelsea', 1991, 4, 27, 12, 4, 0, "Redlands California")

"Chelsea, your birth exoplanet is WASP-3 b orbiting star WASP-3. Based on an eccentricity of 0.00, you prefer stability. With an orbit semi-major axis of 0.03 AU, you are 'close to the action' and constantly influenced by your star's energy, suggesting a very outgoing and active nature. With a birth exoplanet period of 1.85 days, these rapid orbits suggest a fast-paced and reactive thinking style, and with a stellar mass of 1.28 solar masses, you are balanced and nurturing."

In [9]:
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive
from astropy.time import Time
from astropy.coordinates import EarthLocation, AltAz, SkyCoord
import astropy.units as u
from geopy import geocoders  
from geopy.geocoders import Nominatim
from datetime import datetime
import numpy as np

exoplanets_table = NasaExoplanetArchive.query_criteria(table="pscomppars", select="*")



In [10]:


class User(object):
    """
    User class
    """

    def __init__(self, user, citystate, year, month, day, hour, minute, second):
        """
        Args:
            user (str): User's name in the form 'User'
            citystate (str): City and State of birth in the form: 'City State' / 'City Country'
            year (int): Birthyear of User
            month (int): Birthmonth of User
            day (int): Birthday of User
            hour (int): Birthhour of User
            minute (int): Birthminute of User
            second (int): Birthsecond of User
        """

        if not isinstance(user, str):
            raise TypeError("User name must be a string.")

        if not isinstance(citystate, str):
            raise TypeError("City State / City Country must be a string.")

        if not isinstance(year, int):
            raise TypeError("Year must be an integer.")

        if year<=0:
            raise ValueError("Year must be a positive integer.")

        self.user = user

        self.citystate = citystate

        self.year = year
        self.month = month
        self.day = day
        self.hour = hour
        self.minute = minute
        self.second = second
        time = Time(datetime(self.year, self.month, self.day, self.hour, self.minute, self.second))
        self.time = time

        self.closest_object_nasa_table = self.get_closest_table()

    def get_closest_table(self):
        '''
        Get table of closest object

        This method finds the Nasa Exoplanet Archive table of the object which transits nearest birth zenith of the user.

        Args:
            citystate (str): City and State of birth in the form: 'City State'
            year (int): Birthyear of User
            month (int): Birthmonth of User
            day (int): Birthday of User
            hour (int): Birthhour of User
            minute (int): Birthminute of User
            second (int): Birthsecond of User

        Returns:
            astropy.table.table.QTable: table of closest object to birth zenith
        '''
            

        geolocator = Nominatim(user_agent='moeur')
        location = geolocator.geocode(self.citystate)
        date_time = datetime(self.year, self.month, self.day, self.hour, self.minute, self.second)
        self.birth_lat, self.birth_lon = location[1][0], location[1][1]
        zen_ra, zen_dec = self.get_zenith()
        table = exoplanets_table
        coords = SkyCoord(zen_ra, zen_dec, unit=(u.deg, u.deg))
        stars_coords = SkyCoord(table['ra'], table['dec'], unit=(u.deg, u.deg))
        distances = coords.separation(stars_coords)
        closest_index = distances.argmin()
        closest_object = table[closest_index]
        closest_table = NasaExoplanetArchive.query_object(closest_object['pl_name'])
        return closest_table
        
    def get_zenith(self):
        """
        Compute birth zenith

        This method takes latitude and longitude coordinates of the user's birth city and computes from it ra and dec coordinates of the birth zenith.
        
        Returns:
            tuple: (astropy.coordinates.angles.Longitude, astropy.coordinates.angles.Latitude)
        """
        location = EarthLocation(lat=self.birth_lat, lon=self.birth_lon)
        zenith = SkyCoord(alt=90*u.deg, az=0*u.deg, frame=AltAz(obstime=self.time, location=location))
        zenith_radec = zenith.transform_to('icrs')

        return zenith_radec.ra, zenith_radec.dec



    def map_eccentricity_to_trait(self):
        '''
        Map orbital eccentricity to personality trait

        This method assigns a personality trait to the user based on the value of their birth exoplanet's orbital eccentricity.

        Returns:
            string: the personality trait
        '''
        if self.eccentricity == np.nan:
            return ""
        if self.eccentricity == 0:
            return "are perfectly stable"
        elif 0 < self.eccentricity < 0.3:
            return "prefer stability"
        elif 0.3 <= self.eccentricity < 0.6:
            return "are balanced"
        elif 0.6 <= self.eccentricity < 0.9:
            return "prefer excitement"
        else:
            return "embrace change"

    def map_semimajor_axis_to_trait(self):
        '''
        Map orbital semimajor axis to personality trait

        This method assigns a personality trait to the user based on the value of their birth exoplanet's orbital semimajor axis.

        Returns:
            string: the personality trait
        '''
        if self.semimajor_axis == np.nan:
            return ""
        if self.semimajor_axis < 0.1:
            return "are 'close to the action' and constantly influenced by your star's energy, suggesting a very outgoing and active nature"
        elif 0.1 <= self.semimajor_axis < 1:
            return "are still within a region of significant stellar influence, indicating a generally social and engaging character"
        elif 1 <= self.semimajor_axis < 5:
            return "strike a balance between the inner and outer regions, reflecting a well-rounded personality that is equally comfortable in social situations and solitude"
        elif 5 <= self.semimajor_axis < 30:
            return "are farther from the star, implying a more reserved and introspective nature, preferring less direct interaction"
        else:
            return "are on the outskirts, indicating a highly introspective and solitary disposition, thriving in their own space away from the hustle and bustle"

    def map_orbital_period_to_trait(self):
        """
        Map exoplanet system's orbital period to a personality trait (thinking style).

        Returns: 
            str: The personality trait text message.
        """
        if self.period == np.nan:
            return ""
        if self.period < 10:
            return "rapid orbits suggest a fast-paced and reactive thinking style"
        elif 10 <= self.period < 100:
            return "orbits allow for rapid changes and adaptation, indicating an active and adaptive thinking style"
        elif 100 <= self.period < 365:
            return "orbital periods tend to experience balanced conditions, suggesting a balanced and analytical thinking style"
        elif 365 <= self.period < 3650:
            return "planets take longer to orbit their stars, implying a more deliberate and thoughtful approach"
        else:
            return "very long orbital periods embody a reflective and contemplative thinking style"

    def map_stellar_mass_to_trait(self):
        """
        Map exoplanet system's stellar mass to a personality trait.

        Returns:
            str: The personality trait based on the stellar mass.
        """

        if self.stellar_mass == np.nan:
            return ""
        if self.stellar_mass < 0.5:
            return "stable and enduring"
        elif 0.5 <= self.stellar_mass < 1.5:
            return "balanced and nurturing"
        elif 1.5 <= self.stellar_mass < 3:
            return "dynamic and charismatic"
        else:
            return "intense and transformative"
        
    def get_horoscope(self):
        """
        User class method to get the User's horoscope based on User's attributes.

        Returns:
            str: The horoscope message for the User.
        """
        self.planet = self.closest_object_nasa_table['pl_name'][0]
        self.star = self.closest_object_nasa_table['hostname'][0]
        self.eccentricity = np.nanmean(self.closest_object_nasa_table["pl_orbeccen"])
        self.semimajor_axis = np.nanmean(np.asarray(self.closest_object_nasa_table["pl_orbsmax"].value))
        self.period = np.nanmean(np.asarray(self.closest_object_nasa_table["pl_orbper"].value))
        self.stellar_mass = np.nanmean(np.asarray(self.closest_object_nasa_table["st_mass"].value))
        #self.planet_mass = np.nanmean(np.asarray(self.closest_object_nasa_table["pl_bmassj"].value))
        #self.planet_radius = np.nanmean(np.asarray(self.closest_object_nasa_table["pl_radj"].value))
        #self.planet_density = np.nanmean(np.asarray(self.closest_object_nasa_table["pl_dens"].value))
        #self.planet_temp = np.nanmean(np.asarray(self.closest_object_nasa_table["pl_eqt"].value))
        #self.stellar_magnitude = np.nanmean(np.asarray(self.closest_object_nasa_table["st_optmag"].value))
        #self.stellar_radius = np.nanmean(np.asarray(self.closest_object_nasa_table["st_rad"].value))
        #self.stellar_temp = np.nanmean(np.asarray(self.closest_object_nasa_table["st_teff"].value))


        eccentricity_trait = self.map_eccentricity_to_trait()
        axis_trait = self.map_semimajor_axis_to_trait()
        period_trait = self.map_orbital_period_to_trait()
        stellar_mass_trait = self.map_stellar_mass_to_trait()

        message = (f"{self.user}, your birth exoplanet is {self.planet} orbiting star {self.star}. "
                f"Based on an eccentricity of {self.eccentricity:.2f}, you {self.map_eccentricity_to_trait()}. "
                f"With an orbit semi-major axis of {self.semimajor_axis:.2f} AU, you {self.map_semimajor_axis_to_trait()}. "
                f"With a birth exoplanet period of {self.period:.2f} days, these {self.map_orbital_period_to_trait()}, "
                f"and with a stellar mass of {self.stellar_mass:.2f} solar masses, you are {self.map_stellar_mass_to_trait()}.")
        return message

    def map_radius_to_life_suggestion(self):
        '''
        Map planet radius to life suggestion.

        This method assigns a life suggestion to the user based on the value of their birth exoplanet's radius.

        Returns:
            string: the life suggestion
        '''
        if self.radius == np.nan:
            return ""
        if self.radius < 1:
            return "focus on the little things; small steps can lead to big achievements"
        elif 1 <= self.radius < 2:
            return "find balance between ambition and contentment"
        elif 2 <= self.radius < 5:
            return "be bold and take on challenges head-on"
        else:
            return "aim high and don't be afraid to dream big"

    def map_magnitude_to_life_suggestion(self):
        '''
        Map planet magnitude to life suggestion.

        This method assigns a life suggestion to the user based on the value of their birth exoplanet's magnitude.

        Returns:
            string: the life suggestion
        '''
        if self.magnitude == np.nan:
            return ""
        if self.magnitude < 10:
            return "embrace your bright and positive nature"
        elif 10 <= self.magnitude < 15:
            return "find ways to shine even in the dark moments"
        else:
            return "be a guiding light for others around you"

    def map_density_to_life_suggestion(self):
        '''
        Map planet density to life suggestion.

        This method assigns a life suggestion to the user based on the value of their birth exoplanet's density.

        Returns:
            string: the life suggestion
        '''
        if self.density == np.nan:
            return ""
        if self.density < 3:
            return "keep a light-hearted and flexible approach to life"
        elif 3 <= self.density < 5:
            return "balance your seriousness with moments of joy"
        elif 5 <= self.density < 8:
            return "stay grounded and practical in your decisions"
        else:
            return "be resilient and unyielding in the face of challenges"

    def get_life_suggestions(self):
        """
        User class method to get the User's life suggestions based on exoplanet's orbital properties.

        Returns:
            str: The life suggestions message for the User.
        """
        self.planet = self.closest_object_nasa_table['pl_name'][0]
        self.star = self.closest_object_nasa_table['hostname'][0]
        self.radius = np.nanmean(np.asarray(self.closest_object_nasa_table["pl_radj"].value))
        self.magnitude = np.nanmean(np.asarray(self.closest_object_nasa_table["sy_gaiamag"].value)) #no planetary magnitude, so using system's
        self.density = np.nanmean(np.asarray(self.closest_object_nasa_table["pl_dens"].value))
        radius_suggestion = self.map_radius_to_life_suggestion()
        magnitude_suggestion = self.map_magnitude_to_life_suggestion()
        density_suggestion = self.map_density_to_life_suggestion()

        message = (f"{self.user}, your birth exoplanet is {self.planet} orbiting star {self.star}. "
                f"Based on a radius of {self.radius:.2f} Jupiter radii, {radius_suggestion}. "
                f"With a magnitude of {self.magnitude:.2f}, {magnitude_suggestion}. "
                f"And with a density of {self.density:.2f} g/cm³, {density_suggestion}.")
        return message
    
    def get_lucky_numbers(self):
        """
        Generate lucky numbers based on the first two letters of the exoplanet and user names.

        Returns:
            str: A message with the lucky numbers and their corresponding adjectives.
        """
        # Dictionary to map letters to their positions in the alphabet
        letter_to_number = {chr(i + 96): i for i in range(1, 27)}

        # Function to get number from letter
        def letter_number(letter):
            return letter_to_number.get(letter.lower(), 0)

        # Function to get an adjective based on a number
        def number_to_adjective(number):
            adjectives = [
                "amazing", "brave", "creative", "dynamic", "elegant", "fearless", "graceful", "honest",
                "intelligent", "joyful", "kind", "lively", "mighty", "noble", "optimistic", "passionate",
                "quick", "radiant", "strong", "trustworthy", "unique", "vibrant", "wise", "youthful", "zealous"
            ]
            return adjectives[number % len(adjectives)]

        # Get the first two letters of the exoplanet and user names
        planet_letters = self.planet[:2].lower()
        user_letters = self.user[:2].lower()

        # Generate the lucky numbers
        lucky_numbers = [letter_number(letter) for letter in planet_letters + user_letters]

        # Generate the message with lucky numbers and adjectives
        lucky_numbers_message = ", ".join([f"{num} ({number_to_adjective(num)})" for num in lucky_numbers])

        message = f"Lucky numbers: {lucky_numbers_message}."
        return message


In [31]:
geolocator = Nominatim(user_agent='moeur')
location = geolocator.geocode('Quebec City Canada')
date_time = datetime(1998, 9, 4, 17, 0, 0)
birth_ra, birth_dec = location[1][0], location[1][1]
zen_ra, zen_dec = zenith_ra_dec(birth_ra, birth_dec, date_time)
table = NasaExoplanetArchive.query_region(
table="ps", coordinates=SkyCoord(zen_ra * u.deg, zen_dec * u.deg),
radius=24 * u.deg) 
coords = SkyCoord(zen_ra * u.deg, zen_dec * u.deg)
stars_coords = SkyCoord(table['ra'], table['dec'], unit=(u.deg, u.deg))
distances = coords.separation(stars_coords)
closest_index = distances.argmin()
closest_object = table[closest_index]
closest_table = NasaExoplanetArchive.query_object(closest_object['pl_name'])

In [33]:
np.nanmean(np.asarray(closest_table["sy_gaiamag"].value))

9.26803

In [15]:
User('Chelsea', "Redlands California", 1991, 4, 27, 12, 4, 0)

In [11]:
chels = User('Chelsea', "Redlands California", 1991, 4, 27, 12, 4, 0)

In [12]:
chels.get_horoscope()

"Chelsea, your birth exoplanet is WASP-3 b orbiting star WASP-3. Based on an eccentricity of 0.00, you prefer stability. With an orbit semi-major axis of 0.03 AU, you are 'close to the action' and constantly influenced by your star's energy, suggesting a very outgoing and active nature. With a birth exoplanet period of 1.85 days, these rapid orbits suggest a fast-paced and reactive thinking style, and with a stellar mass of 1.28 solar masses, you are balanced and nurturing."

In [13]:
chels.get_life_suggestions()

'Chelsea, your birth exoplanet is WASP-3 b orbiting star WASP-3. Based on a radius of 1.39 Jupiter radii, find balance between ambition and contentment. With a magnitude of 10.45, find ways to shine even in the dark moments. And with a density of 0.94 g/cm³, keep a light-hearted and flexible approach to life.'

In [14]:
chels.get_lucky_numbers()

'Lucky numbers: 23 (youthful), 1 (brave), 3 (dynamic), 8 (intelligent).'

In [19]:
exoplanets_table

pl_name,pl_letter,hostname,hd_name,hip_name,tic_id,disc_pubdate,disc_year,discoverymethod,disc_locale,disc_facility,disc_instrument,disc_telescope,disc_refname,ra,rastr,dec,decstr,glon,glat,elon,elat,ra_reflink,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbperstr,pl_orbper_reflink,pl_orblpererr1,pl_orblper,pl_orblpererr2,pl_orblperlim,pl_orblperstr,pl_orblper_reflink,pl_orbsmax,pl_orbsmaxerr1,pl_orbsmaxerr2,pl_orbsmaxlim,pl_orbsmaxstr,pl_orbsmax_reflink,pl_angsep,pl_angseperr1,pl_angseperr2,pl_angseplim,pl_angsepstr,pl_angsep_reflink,pl_orbincl,pl_orbinclerr1,pl_orbinclerr2,pl_orbincllim,pl_orbinclstr,pl_orbincl_reflink,pl_orbtper,pl_orbtpererr1,pl_orbtpererr2,pl_orbtperlim,pl_orbtperstr,pl_orbtper_reflink,pl_orbeccen,pl_orbeccenerr1,pl_orbeccenerr2,pl_orbeccenlim,pl_orbeccenstr,pl_orbeccen_reflink,pl_eqt,pl_eqterr1,pl_eqterr2,pl_eqtlim,pl_eqtstr,pl_eqt_reflink,pl_occdep,pl_occdeperr1,pl_occdeperr2,pl_occdeplim,pl_occdepstr,pl_occdep_reflink,pl_insol,pl_insolerr1,pl_insolerr2,pl_insollim,pl_insolstr,pl_insol_reflink,pl_dens,pl_denserr1,pl_denserr2,pl_denslim,pl_densstr,pl_dens_reflink,pl_trandep,pl_trandeperr1,pl_trandeperr2,pl_trandeplim,pl_trandepstr,pl_trandep_reflink,pl_tranmid,pl_tranmiderr1,pl_tranmiderr2,pl_tranmidlim,pl_tranmidstr,sy_pmdec,sy_pmdecerr1,sy_pmdecerr2,sy_pmdecstr,sy_plx,sy_plxerr1,sy_plxerr2,sy_plxstr,sy_plx_reflink,sy_dist,sy_disterr1,sy_disterr2,sy_diststr,sy_dist_reflink,sy_bmag,sy_bmagerr1,sy_bmagerr2,sy_bmagstr,sy_bmag_reflink,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_vmagstr,sy_vmag_reflink,sy_jmag,sy_jmagerr1,sy_jmagerr2,sy_jmagstr,sy_jmag_reflink,sy_hmag,sy_hmagerr1,sy_hmagerr2,sy_hmagstr,sy_hmag_reflink,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_kmagstr,sy_kmag_reflink,sy_umag,sy_umagerr1,sy_umagerr2,sy_umagstr,sy_umag_reflink,sy_rmag,sy_rmagerr1,sy_rmagerr2,sy_rmagstr,sy_rmag_reflink,sy_imag,sy_imagerr1,sy_imagerr2,sy_imagstr,sy_imag_reflink,sy_zmag,sy_zmagerr1,sy_zmagerr2,sy_zmagstr,sy_zmag_reflink,sy_w1mag,sy_w1magerr1,sy_w1magerr2,sy_w1magstr,sy_w1mag_reflink,sy_w2mag,sy_w2magerr1,sy_w2magerr2,sy_w2magstr,sy_w2mag_reflink,sy_w3mag,sy_w3magerr1,sy_w3magerr2,sy_w3magstr,sy_w3mag_reflink,sy_w4mag,sy_w4magerr1,sy_w4magerr2,sy_w4magstr,sy_w4mag_reflink,sy_gmag,sy_gmagerr1,sy_gmagerr2,sy_gmagstr,sy_gmag_reflink,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,sy_gaiamagstr,sy_gaiamag_reflink,sy_tmag,sy_tmagerr1,sy_tmagerr2,sy_tmagstr,sy_tmag_reflink,pl_controv_flag,pl_orbtper_systemref,pl_tranmid_systemref,st_metratio,st_spectype,st_spectype_reflink,sy_kepmag,sy_kepmagerr1,sy_kepmagerr2,sy_kepmagstr,sy_kepmag_reflink,st_rotp,st_rotperr1,st_rotperr2,st_rotplim,st_rotpstr,st_rotp_reflink,pl_projobliq,pl_projobliqerr1,pl_projobliqerr2,pl_projobliqlim,pl_projobliqstr,pl_projobliq_reflink,gaia_id,cb_flag,pl_tranmid_reflink,pl_trandur,pl_trandurerr1,pl_trandurerr2,pl_trandurlim,pl_trandurstr,pl_trandur_reflink,pl_rvamp,pl_rvamperr1,pl_rvamperr2,pl_rvamplim,pl_rvampstr,pl_rvamp_reflink,pl_radj,pl_radjerr1,pl_radjerr2,pl_radjlim,pl_radjstr,pl_radj_reflink,pl_rade,pl_radeerr1,pl_radeerr2,pl_radelim,pl_radestr,pl_rade_reflink,pl_ratror,pl_ratrorerr1,pl_ratrorerr2,pl_ratrorlim,pl_ratrorstr,pl_ratror_reflink,pl_ratdor,pl_trueobliq,pl_trueobliqerr1,pl_trueobliqerr2,pl_trueobliqlim,pl_trueobliqstr,pl_trueobliq_reflink,sy_icmag,sy_icmagerr1,sy_icmagerr2,sy_icmagstr,sy_icmag_reflink,dkin_flag,pl_ratdorerr1,pl_ratdorerr2,pl_ratdorlim,pl_ratdorstr,pl_ratdor_reflink,pl_imppar,pl_impparerr1,pl_impparerr2,pl_impparlim,pl_impparstr,pl_imppar_reflink,pl_bmassj,pl_bmassjerr1,pl_bmassjerr2,pl_bmassjlim,pl_bmassjstr,pl_bmassj_reflink,pl_bmasse,pl_bmasseerr1,pl_bmasseerr2,pl_bmasselim,pl_bmassestr,pl_bmasse_reflink,pl_bmassprov,st_teff,st_tefferr1,st_tefferr2,st_tefflim,st_teffstr,st_teff_reflink,st_met,st_meterr1,st_meterr2,st_metlim,st_metstr,st_met_reflink,st_radv,st_radverr1,st_radverr2,st_radvlim,st_radvstr,st_radv_reflink,st_vsin,st_vsinerr1,st_vsinerr2,st_vsinlim,st_vsinstr,st_vsin_reflink,st_lum,st_lumerr1,st_lumerr2,st_lumlim,s

In [21]:
import pandas as pd
df = exoplanets_table.to_pandas()

# Print the column names to examine them
print(df.columns)

# Function to find columns that contain the keyword "mag" or "magnitude"
def find_magnitude_columns(df):
    keywords = ['mag', 'magnitude']
    magnitude_columns = [col for col in df.columns if any(keyword in col.lower() for keyword in keywords)]
    return magnitude_columns

# Get the magnitude columns
magnitude_columns = find_magnitude_columns(df)
print("Potential magnitude columns:", magnitude_columns)

Index(['pl_name', 'pl_letter', 'hostname', 'hd_name', 'hip_name', 'tic_id',
       'disc_pubdate', 'disc_year', 'discoverymethod', 'disc_locale',
       ...
       'sy_pmra', 'sy_pmraerr1', 'sy_pmraerr2', 'sy_pmrastr', 'x', 'y', 'z',
       'htm20', 'sky_coord.ra', 'sky_coord.dec'],
      dtype='object', length=381)
Potential magnitude columns: ['sy_bmag', 'sy_bmagerr1', 'sy_bmagerr2', 'sy_bmagstr', 'sy_bmag_reflink', 'sy_vmag', 'sy_vmagerr1', 'sy_vmagerr2', 'sy_vmagstr', 'sy_vmag_reflink', 'sy_jmag', 'sy_jmagerr1', 'sy_jmagerr2', 'sy_jmagstr', 'sy_jmag_reflink', 'sy_hmag', 'sy_hmagerr1', 'sy_hmagerr2', 'sy_hmagstr', 'sy_hmag_reflink', 'sy_kmag', 'sy_kmagerr1', 'sy_kmagerr2', 'sy_kmagstr', 'sy_kmag_reflink', 'sy_umag', 'sy_umagerr1', 'sy_umagerr2', 'sy_umagstr', 'sy_umag_reflink', 'sy_rmag', 'sy_rmagerr1', 'sy_rmagerr2', 'sy_rmagstr', 'sy_rmag_reflink', 'sy_imag', 'sy_imagerr1', 'sy_imagerr2', 'sy_imagstr', 'sy_imag_reflink', 'sy_zmag', 'sy_zmagerr1', 'sy_zmagerr2', 'sy_zmagstr', 'sy_z